# CRAWL DATA FROM LAZADA

## 1. Setting selenium and initialize Chrome


In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from selenium import webdriver
import urllib.request
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from time import sleep
import random
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException, ElementNotInteractableException

In [2]:
main_link = 'https://www.lazada.vn/catalog/?q=laptop'
path = 'chromedriver.exe'

# # Customize chrome display
chrome_options = Options()
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-notifications')
chrome_options.add_argument('--disable-infobars')

service = Service(executable_path=path)
driver = webdriver.Chrome(service=service, options=chrome_options)

driver.get(main_link)

In [3]:
def web_driver():
    options = webdriver.ChromeOptions()
    options.add_argument("--verbose")
    options.add_argument('--no-sandbox')
    options.add_argument('--headless')
    options.add_argument('--disable-gpu')
    options.add_argument("--window-size=1920, 1200")
    options.add_argument('--disable-dev-shm-usage')
    driver = webdriver.Chrome(options = options)
    return driver

In [4]:
driver = web_driver()
driver.get('https://www.lazada.vn/catalog/?q=laptop')
sleep(random.randint(3,5))

## 2. Crawl data using class

In [5]:
# ================================ GET links/title
elems = driver.find_elements(By.CSS_SELECTOR , ".RfADt [href]")
Title = [elem.text for elem in elems]
Links = [elem.get_attribute('href') for elem in elems]

In [6]:
# ================================ GET type
elems = driver.find_elements(By.CSS_SELECTOR , ".JrAyI")
Type = [elem.text for elem in elems]
if len(Type) == 1:
    Type = Type * len(Title)

In [7]:
# ================================ GET  price sale
elems = driver.find_elements(By.CSS_SELECTOR , ".ooOxS")
Price_sale = [elem.text for elem in elems]

In [8]:
# ================================ GET sale off 
elems = driver.find_elements(By.CSS_SELECTOR , ".WNoq3")
Sale_off = [elem.text for elem in elems]

In [9]:
# ================================ GET Location
elems = driver.find_elements(By.CSS_SELECTOR , ".oa6ri")
Location = [elem.text for elem in elems]

In [12]:
df1 = pd.DataFrame(list(zip(Type ,Title, Links, Price_sale, Sale_off, Location)),
                   columns = ['Type', 'Title', 'Link', 'Price_sale', 'Sale_off', 'Location'])
df1['Index_']= np.arange(1, len(df1) + 1)

In [15]:
df1.to_csv('df1_demo.csv', encoding='utf-8', index=False)

In [13]:
print(df1)

      Type                                              Title  \
0   laptop  Máy Tính Xách Tay Bàn Phím Bao Gồm, Màng Bảo V...   
1   laptop  Laptop Dell Inspiron 7567 máy tính giá rẻ(Core...   
2   laptop  Laptop cũ nhiều lựa chọn Pentium đến i3 i5 i7 ...   
3   laptop  Laptop core 2 giá rẻ, ram 4G, HDD 250G, đầy đủ...   
4   laptop  Máy tính laptop cấu hình đời cao chip I3 siêu ...   
5   laptop                               Laptop mini hàng đẹp   
6   laptop  Thành lý máy tính sách tay laptop học tập sử d...   
7   laptop  Laptop game và đồ họa giá tốt HP 6470b Core i5...   
8   laptop  Lenovo Thinkpad / Tosiba / Dell Core i5 / RAM ...   
9   laptop  Laptop cũ giá rẻ - Máy tính zin ram 4GB ổ 250g...   
10  laptop  Thanh lý Laptop Toshiba, core I5, ram 4Gb, ô c...   
11  laptop  Laptop Dell Latitude E7250 màn 12.5 - Core i7 ...   
12  laptop  Thanh lý Laptop Lenovo, Pentum (R), Giá bán 1,...   
13  laptop  Laptop Dell Latitude E5480 màn 14 inch Ram 8G ...   
14  laptop  Laptop Dell L

## 3. Crawl data using xpath

In [14]:
# ================================ GET total sold/ preview
Total_sold, Preview, Index_ = [], [], []
for i in range(1, len(Title) + 1):
    try:
        total = driver.find_element("xpath", f"/html/body/div[3]/div/div[2]/div[1]/div/div[1]/div[2]/div[{i}]/div/div/div[2]/div[5]/span[1]/span[1]")
        Total_sold.append(total.text)
    except NoSuchElementException:
        Total_sold.append(None)  # Đúng danh sách và phần tử
    try:
        prev = driver.find_element("xpath", f"/html/body/div[3]/div/div[2]/div[1]/div/div[1]/div[2]/div[{i}]/div/div/div[2]/div[5]/div/span")
        Preview.append(prev.text)
    except NoSuchElementException:
        Preview.append(None)  # Đúng danh sách và phần tử
    try:
        Index_.append(i)
    except NoSuchElementException:
        Preview.append(None)  # Đúng danh sách và phần tử

In [16]:
df2 = pd.DataFrame(list(zip(Index_ , Total_sold, Preview)), columns = ['Index_', 'Total_sold', 'Preview'])

In [17]:
df2.to_csv('df2_demo.csv', encoding='utf-8', index=False)

In [18]:
print(df2)

    Index_  Total_sold Preview
0        1  311 Đã bán    (67)
1        2        None    None
2        3   81 Đã bán    (28)
3        4   22 Đã bán    (10)
4        5        None     (3)
5        6    9 Đã bán     (2)
6        7   15 Đã bán     (2)
7        8   83 Đã bán    (40)
8        9   18 Đã bán     (4)
9       10   24 Đã bán    (11)
10      11   25 Đã bán    (10)
11      12   33 Đã bán    (18)
12      13    9 Đã bán     (1)
13      14   73 Đã bán    (31)
14      15   64 Đã bán    (28)
15      16   14 Đã bán     (2)
16      17  351 Đã bán   (108)
17      18  118 Đã bán    (35)
18      19    8 Đã bán     (3)
19      20  126 Đã bán    (29)
20      21   13 Đã bán     (9)
21      22   15 Đã bán     (3)
22      23   75 Đã bán    (21)
23      24   38 Đã bán    (12)
24      25    9 Đã bán     (2)
25      26   97 Đã bán    (23)
26      27  261 Đã bán   (133)
27      28   69 Đã bán    (28)
28      29  127 Đã bán    (27)
29      30   40 Đã bán     (7)
30      31   33 Đã bán     (9)
31      

In [19]:
df3 = df1.merge(df2, how='left', left_on='Index_', right_on='Index_')

In [20]:
print(df3)

      Type                                              Title  \
0   laptop  Máy Tính Xách Tay Bàn Phím Bao Gồm, Màng Bảo V...   
1   laptop  Laptop Dell Inspiron 7567 máy tính giá rẻ(Core...   
2   laptop  Laptop cũ nhiều lựa chọn Pentium đến i3 i5 i7 ...   
3   laptop  Laptop core 2 giá rẻ, ram 4G, HDD 250G, đầy đủ...   
4   laptop  Máy tính laptop cấu hình đời cao chip I3 siêu ...   
5   laptop                               Laptop mini hàng đẹp   
6   laptop  Thành lý máy tính sách tay laptop học tập sử d...   
7   laptop  Laptop game và đồ họa giá tốt HP 6470b Core i5...   
8   laptop  Lenovo Thinkpad / Tosiba / Dell Core i5 / RAM ...   
9   laptop  Laptop cũ giá rẻ - Máy tính zin ram 4GB ổ 250g...   
10  laptop  Thanh lý Laptop Toshiba, core I5, ram 4Gb, ô c...   
11  laptop  Laptop Dell Latitude E7250 màn 12.5 - Core i7 ...   
12  laptop  Thanh lý Laptop Lenovo, Pentum (R), Giá bán 1,...   
13  laptop  Laptop Dell Latitude E5480 màn 14 inch Ram 8G ...   
14  laptop  Laptop Dell L

In [21]:
df3.to_csv('df3_demo.csv', encoding='utf-8', index=False)

In [22]:
print(Links[0])

https://www.lazada.vn/products/may-tinh-xach-tay-ban-phim-bao-gom-mang-bao-ve-ban-phim-cho-14-inch-latitude-3400-a3430-dell-inspiron-14-3000-5468-5000-5488-5480-i1496850993.html


## 4. Crawl data using class in link item

In [27]:
Price_original, Ship_price, Return_exchange, Sale_rating, Ship_on_time, Chat_response = [], [], [], [], [], []
 
for link in Links:
    driver.get(link)

    # ================================ GET price original
    elems = driver.find_elements(By.CSS_SELECTOR, ".origin-block .notranslate")
    one_price_original = [elem.text for elem in elems]
    Price_original.append(one_price_original)

    # ================================ GET ship price
    elems = driver.find_elements(By.CSS_SELECTOR, ".delivery-option-item__shipping-fee")
    one_ship_prices = [elem.text for elem in elems]
    # if len(one_ship_prices) != 0: 
    one_ship_price = one_ship_prices[0]
    Ship_price.append(one_ship_price)

    # ================================ GET return
    elems = driver.find_elements(By.CSS_SELECTOR, ".warranty__option-item .delivery-option-item__body .delivery-option-item__info .delivery-option-item__title")
    one_return_exchanges = [elem.text for elem in elems]
    one_return_exchange = one_return_exchanges[0]
    Return_exchange.append(one_return_exchange)

    # ================================ GET sale rating/ shop on tim/ chat response
    elems = driver.find_elements(By.CSS_SELECTOR, ".info-content .seller-info-value")
    percent_data = [elem.text for elem in elems]

    for i, percent in enumerate(percent_data):
        if i == 0:
            one_sale_rating = percent # sale rating
        elif i == 1:
            one_ship_on_time = percent # shop on time 
        elif i == 2:
            one_chat_response = percent # chat response

    Sale_rating.append(one_sale_rating)
    Ship_on_time.append(one_ship_on_time)
    Chat_response.append(one_chat_response)

In [28]:
One_star, Two_star, Three_star, Four_star, Five_star = [], [], [], [], []
count = 0

for link in Links: 
    driver.get(link)

    # ================================ GET star data
    elems = driver.find_elements(By.CSS_SELECTOR, ".percent")
    stars = [elem.text for elem in elems]
    print(count)
    # Kiểm tra và thêm các giá trị sao một cách an toàn
    One_star.append(stars[4] if len(stars) > 4 else 'N/A')
    Two_star.append(stars[3] if len(stars) > 3 else 'N/A')
    Three_star.append(stars[2] if len(stars) > 2 else 'N/A')
    Four_star.append(stars[1] if len(stars) > 1 else 'N/A')
    Five_star.append(stars[0] if len(stars) > 0 else 'N/A')
    count += 1
    if count >= 40:
        break

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39


In [29]:
print(One_star)
print(Two_star)
print(Three_star)
print(Four_star)
print(Five_star)

['11', '0', '0', '2', '1', '0', '1', '0', '0', '0', '3', '0', '0', '0', '0', '0', '3', '0', '0', '0', '3', '0', '0', '0', '0', '1', '0', '0', '1', '1', '1', '1', '0', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A']
['1', '0', '0', '1', '0', '0', '0', '1', '0', '0', '1', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A']
['3', '0', '0', '1', '0', '0', '0', '0', '0', '0', '2', '0', '1', '0', '0', '1', '0', '0', '0', '0', '1', '0', '0', '0', '0', '1', '0', '0', '0', '2', '0', '0', '0', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A']
['5', '0', '0', '3', '0', '0', '0', '1', '0', '6', '12', '1', '0', '0', '1', '0', '1', '0', '0', '0', '0', '0', '0', '0', '0', '0', '2', '0', '0', '0', '0', '2', '0', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A']
['47', '0', '9', '21', '2', '3', '1', '9', '2', '34', '761', '9', '30', '28', '40', '1', '104', '10', '12', '1', '20', '28', '0', '27', '7

In [30]:
df4 = pd.DataFrame(list(zip(Price_original , Ship_price, Return_exchange, Sale_rating, Ship_on_time, Chat_response, One_star, Two_star, Three_star, Four_star, Five_star)), 
                   columns = ['price_original', 'ship_price','Return', 'Sale_rating', 'Ship_on_time', 'Chat_response', 'One_star', 'Two_star', 'Three_star', 'Four_star', 'Five_star'])
df4['Index_']= np.arange(1, len(df4) + 1)

In [31]:
df4.to_csv('df4_demo.csv', encoding='utf-8', index=False)

In [32]:
print(df4)

    price_original ship_price                            Return  \
0       [47.000 ₫]                      7 ngày trả hàng dễ dàng   
1    [6.400.000 ₫]   Miễn phí  3 ngày trả hàng cho Nhà bán hàng   
2    [3.200.000 ₫]   52.400 ₫  3 ngày trả hàng cho Nhà bán hàng   
3               []  198.200 ₫  3 ngày trả hàng cho Nhà bán hàng   
4    [1.499.999 ₫]   70.000 ₫  3 ngày trả hàng cho Nhà bán hàng   
5    [1.399.999 ₫]   80.200 ₫  3 ngày trả hàng cho Nhà bán hàng   
6    [1.499.999 ₫]  100.600 ₫  3 ngày trả hàng cho Nhà bán hàng   
7    [2.300.000 ₫]   Miễn phí  3 ngày trả hàng cho Nhà bán hàng   
8               []   40.300 ₫  3 ngày trả hàng cho Nhà bán hàng   
9               []   74.400 ₫  3 ngày trả hàng cho Nhà bán hàng   
10              []   16.500 ₫  3 ngày trả hàng cho Nhà bán hàng   
11              []   32.200 ₫  3 ngày trả hàng cho Nhà bán hàng   
12   [3.998.000 ₫]   56.800 ₫  3 ngày trả hàng cho Nhà bán hàng   
13   [3.598.000 ₫]   56.800 ₫  3 ngày trả hàng cho Nhà bán hàn

In [33]:
df5 = df3.merge(df4, how='left', left_on='Index_', right_on='Index_')

In [34]:
df5.to_csv('df5_demo.csv', encoding='utf-8', index=False)

In [35]:
print(df5)

                                                Title  \
0   Máy Tính Xách Tay Bàn Phím Bao Gồm, Màng Bảo V...   
1   Laptop Dell Inspiron 7567 máy tính giá rẻ(Core...   
2   Laptop HP Probook / Intel Core i5 ~ 2.4Ghz / R...   
3   Laptop cũ nhiều lựa chọn Pentium đến i3 i5 i7 ...   
4                                Laptop mini hàng đẹp   
5   Máy tính laptop cấu hình đời cao chip I3 siêu ...   
6   Thành lý máy tính sách tay laptop học tập sử d...   
7   Laptop cũ giá rẻ - Máy tính zin ram 4GB ổ 250g...   
8   Bán Xác Laptop Acer 4349, không lên hình. Giá ...   
9   Laptop game và đồ họa giá tốt HP 6470b Core i5...   
10  Ổ cứng Laptop HDD, SSD M2 1TB, 750GB, 640Gb, 5...   
11  Thanh lý Laptop Toshiba, core I5, ram 4Gb, ô c...   
12  Laptop Dell Latitude E5480 màn 14 inch Ram 8G ...   
13  Laptop Dell Latitude E7270 ram 8G SSD 256G - C...   
14  Laptop cũ giá rẻ nhiều hãng core i3 i5 ram 4gb...   
15  Laptop HP 4730S i7 , SSD120G , Ram 8G Hàng Nhậ...   
16  Laptop HP 15s-fq5228TU - 8U